In [18]:
import numpy as np
import zarr
import os

from skimage.data import binary_blobs
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

In [22]:

path = "./data/aymanns/test_zarr/test_ngff_image.zarr"
os.makedirs(path, exist_ok=True)

In [23]:
mean_val=10
size_xy = 512
size_z = 100
rng = np.random.default_rng(0)
data = rng.poisson(mean_val, size=(size_z, size_xy, size_xy)).astype(np.uint8)

In [24]:
# write the image data
store = parse_url(path, mode="w").store
root = zarr.group(store=store)
write_image(image=data, group=root, axes="zyx", storage_options=dict(chunks=(1, size_xy, size_xy)))
# optional rendering settings
root.attrs["omero"] = {
    "channels": [{
        "color": "00FFFF",
        "window": {"start": 0, "end": 20, "min": 0, "max": 255},
        "label": "random",
        "active": True,
    }]
}

In [25]:
# add labels...
blobs = binary_blobs(length=size_xy, volume_fraction=0.1, n_dim=3).astype('int8')
blobs2 = binary_blobs(length=size_xy, volume_fraction=0.1, n_dim=3).astype('int8')
# blobs will contain values of 1, 2 and 0 (background)
blobs += 2 * blobs2

# label.shape is (size_xy, size_xy, size_xy), Slice to match the data
label = blobs[:size_z, :, :]

In [26]:
# write the labels to /labels
labels_grp = root.create_group("labels")
# the 'labels' .zattrs lists the named labels data
label_name = "blobs"
labels_grp.attrs["labels"] = [label_name]
label_grp = labels_grp.create_group(label_name)
# need 'image-label' attr to be recognized as label
label_grp.attrs["image-label"] = {
    "colors": [
        {"label-value": 1, "rgba": [255, 0, 0, 255]},
        {"label-value": 2, "rgba": [0, 255, 0, 255]},
        {"label-value": 3, "rgba": [255, 255, 0, 255]}
    ]
}

In [27]:
write_image(label, label_grp, axes="zyx")

[]